In [5]:
import pickle5 as pickle
import numpy as np
import matplotlib.pyplot as plt


In [6]:
# open the data
pkl_file = open("/Users/liyurun/DataAnalysis_Seino/data/distance_20200613/distance_data_20200613.pkl","rb")
data = pickle.load(pkl_file)
pkl_file.close()

In [9]:
# test for 3rd driver's location.
loc = data[2][0][2]
loc

array([[ 35.3537817, 136.6055703],
       [ 35.3550777, 136.6069943],
       [ 35.3596804, 136.6041777],
       [ 35.3604738, 136.6073637],
       [ 35.3644233, 136.6153787],
       [ 35.3625716, 136.6172645],
       [ 35.375565 , 136.6207   ],
       [ 34.6797165, 135.5338832],
       [ 35.3661958, 136.6178395],
       [ 35.3663734, 136.6194059],
       [ 35.3648297, 136.617386 ],
       [ 35.3640365, 136.6194888],
       [ 35.3648526, 136.6187082]])

In [4]:
def cal_dis(lat1, lon1, lat2, lon2):
    lat1 = (math.pi / 180) * lat1
    lat2 = (math.pi / 180) * lat2
    lon1 = (math.pi / 180) * lon1
    lon2 = (math.pi / 180) * lon2

    R = 6378.137
    if (lat1 != lat2) & (lon1 != lon2) :
        d = math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)) * R * 1000
    else:
        d = 0
    return d

In [42]:
# 距离函数
def distance(vec1, vec2):
    return np.linalg.norm(np.array(vec1) - np.array(vec2))
def init_chromos(start=-1, gene_len=-1):
    chroms = []  # 所有种群
    for i in range(population_num):
        gene = list(range(gene_len))  # 染色体基因编码
        np.random.shuffle(gene)
        # 调换染色体的头部基因为给定的基因
        if start != -1:
            for j, g in enumerate(gene):
                if g == start:
                    gene[0], gene[j] = gene[j], gene[0]
        chroms.append(gene)
    return chroms
# 适应度打分函数，返回该染色体的适应度(优秀)分数
def calc_fin_ness(citys, gens):
    gens = np.copy(gens)
    gens = np.append(gens, gens[0])  # 在染色体的末尾添加上头部基因
    D = np.sum([distance(citys[gens[i]], citys[gens[i+1]]) for i in range(len(gens) - 1)])
    return 1.0 / D
# 赌徒转盘(精英染色体被选中的概率与适应度函数打分的结果成正比)
def roulette_gambler(fit_pros, chroms):
    pick = np.random.random()
    for j in range(len(chroms)):
        pick -= fit_pros[j]
        if pick <= 0:
            return j
    return 0
def choice(citys, chroms):
    n = len(chroms)
    fit_pros = []
    [fit_pros.append(calc_fin_ness(citys, chroms[i])) for i in range(n)]
    choice_gens = []
    for i in range(n):
        j = roulette_gambler(fit_pros, chroms)  # 采用赌徒转盘选择出一个更好的染色体
        choice_gens.append(chroms[j])  # 选中一个染色体
    for i in range(n):
        chroms[i] = choice_gens[i]  # 优胜劣汰，替换出更精英的染色体
    return chroms
# 染色体交叉操作(两个染色体互相杂交基因片段，用于产生新的染色体，影响全局寻优能力)
def cross(chroms):
    gens_len = len(chroms[0])
    move = 0  # 当前基因移动的位置
    while move < gens_len - 1:
        cur_pro = np.random.random()  # 决定是否进行交叉操作
        # 本次不进行交叉操作
        if cur_pro > cross_pro:
            move += 2
            continue
        parent1, parent2 = move, move + 1  # 准备杂交的两个染色体(种群)
        index1 = np.random.randint(1, gens_len - 2)
        index2 = np.random.randint(index1, gens_len - 2)
        if index1 == index2:
            continue
        temp_gen1 = chroms[parent1][index1:index2+1]  # 交叉的基因片段1
        temp_gen2 = chroms[parent2][index1:index2+1]  # 交叉的基因片段2
        # 杂交插入染色体片段
        temp_parent1, temp_parent2 = np.copy(chroms[parent1]).tolist(), np.copy(chroms[parent2]).tolist()
        temp_parent1[index1:index2+1] = temp_gen2
        temp_parent2[index1:index2+1] = temp_gen1
        # 消去冲突
        pos = index1 + len(temp_gen1)  # 插入杂交基因片段的结束位置
        conflict1_ids, conflict2_ids = [], []
        [conflict1_ids.append(i) for i, v in enumerate(temp_parent1) if v in temp_parent1[index1:pos]
         and i not in list(range(index1, pos))]
        [conflict2_ids.append(i) for i, v in enumerate(temp_parent2) if v in temp_parent2[index1:pos]
         and i not in list(range(index1, pos))]
        for i, j in zip(conflict1_ids, conflict2_ids):
            temp_parent1[i], temp_parent2[j] = temp_parent2[j], temp_parent1[i]
        chroms[parent1] = temp_parent1
        chroms[parent2] = temp_parent2
        move += 2
    return chroms
# 变异操作(随机调换单个染色体的基因位置)
def mutation(chroms):
    n = len(chroms)
    gens_len = len(chroms[0])
    for i in range(n):
        cur_pro = np.random.random()  # 决定是否进行变异操作
        # 本次不进行变异操作
        if cur_pro > mutation_pro:
            continue
        index1 = np.random.randint(1, gens_len - 2)
        index2 = np.random.randint(1, gens_len - 2)
        chroms[i][index1], chroms[i][index2] = chroms[i][index2], chroms[i][index1]
    return chroms
# 逆转操作(让单个染色体逆转之后变得更加优秀)
def reverse(citys, chroms):
    n = len(chroms)
    gens_len = len(chroms[0])
    for i in range(n):
        flag = 0
        while flag == 0:
            index1 = np.random.randint(1, gens_len - 2)
            index2 = np.random.randint(index1, gens_len - 2)
            if index1 == index2:
                continue
            temp_chrom = np.copy(chroms[i])
            temp_chrom = temp_chrom.tolist()
            temp_gen = temp_chrom[index1:index2+1]
            temp_gen.reverse()
            temp_chrom[index1:index2 + 1] = temp_gen
            fit_score1 = calc_fin_ness(citys, chroms[i])
            fit_score2 = calc_fin_ness(citys, temp_chrom)
            # 说明经过逆转操作后的染色体将变得更加优秀
            if fit_score2 > fit_score1:
                chroms[i] = temp_chrom  # 更新染色体为逆转染色体
            flag = 1
    return chroms
# 可视化画出哈密顿回路
def draw_H(citys, best_gens):
    H_path = []
    [H_path.append((cur_gen, best_gens[i+1])) for i, cur_gen in enumerate(best_gens[:-1])]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.xlim(0, 7)
    plt.ylim(0, 7)
    for (from_, to_) in H_path:
        p1 = plt.Circle(citys[from_], color='red')
        p2 = plt.Circle(citys[to_], color='green')
        ax.add_patch(p1)
        ax.add_patch(p2)
        ax.plot((citys[from_][0], citys[to_][0]), (citys[from_][1], citys[to_][1]), color='red')
        ax.annotate(s=chr(97 + to_), xy=citys[to_], xytext=(-8, -4), textcoords='offset points', fontsize=20)
    ax.axis('equal')
    ax.grid()
    plt.show()

In [43]:
max_evolution_num = 200  # 最大进化代数
population_num = 100  # 种群数目
cross_pro = 0.6  # 交叉概率
mutation_pro = 0.1  # 变异概率
citys = loc  # 城市坐标
best_gens = [-1 for _ in range(len(citys))]  # 精英染色体(基因排列)
min_distance = np.inf  # 最短路径长度
best_fit_index = 0  # 最短路径出现的代数
start = 0  # 种群的初始位置

In [44]:
max_evolution_num = 200  # 最大进化代数
population_num = 100  # 种群数目
cross_pro = 0.6  # 交叉概率
mutation_pro = 0.1  # 变异概率
citys = loc  # 城市坐标
best_gens = [-1 for _ in range(len(citys))]  # 精英染色体(基因排列)
min_distance = np.inf  # 最短路径长度
best_fit_index = 0  # 最短路径出现的代数
start = 0  # 种群的初始位置
# 开始达尔文进化
for step in range(max_evolution_num):
    distance_arr = []  # 每一个染色体的总路程数组
    chroms = init_chromos(start=start, gene_len=len(citys))  # 种群初始化，得到所有种群
    chroms = choice(citys, chroms)  # 选择操作，选择出每个种群的精英染色体
    chroms = cross(chroms)  # 交叉操作，两个染色体互相杂交产生新的染色体
    chroms = mutation(chroms)  # 变异操作，单个染色体变异
    chroms = reverse(citys, chroms)  # 变异操作，单个染色体变得更加优秀
    [distance_arr.append(1.0 / calc_fin_ness(citys, chroms[i])) for i in range(len(chroms))]
    best_gens_idx = np.argmin(distance_arr)  # 找到最短的路径位置，对应于精英染色体位置
    if distance_arr[best_gens_idx] < min_distance:
        min_distance = distance_arr[best_gens_idx]  # 更新最短路径
        best_gens = chroms[best_gens_idx]  # 更新精英染色体
        best_gens.append(start)
        best_fit_index += 1  # 更新最短路径出现的代数
print('通过{}代的基因进化，精英染色体出现在第{}代，基因序列为：'.format(max_evolution_num, best_fit_index))
[print(chr(97 + v), end=',' if i < len(best_gens) - 1 else '\n') for i, v in enumerate(best_gens)]
print('精英染色体映射的最短路径为：{}'.format(min_distance))
draw_H(citys, best_gens)

通过200代的基因进化，精英染色体出现在第6代，基因序列为：
a,b,k,f,e,m,l,j,g,i,d,c,h,a
精英染色体映射的最短路径为：2.596274175464136


/opt/miniconda3/envs/jupyter_env/lib/python3.7/site-packages/ipykernel_launcher.py:125: MatplotlibDeprecationWarning: The 's' parameter of annotate() has been renamed 'text' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.


In [21]:
a=[[1,1],[2,2]]
np.linalg.norm([np.array(a[0])-np.array(a[1])])

1.4142135623730951

In [36]:
listloc=[]
for i in range(len(loc)):
    listloc.append(tuple(loc[i]))

In [35]:
tuple(loc[0])

(35.3537817, 136.6055703)

In [37]:
listloc

[(35.3537817, 136.6055703),
 (35.3550777, 136.6069943),
 (35.3596804, 136.6041777),
 (35.3604738, 136.6073637),
 (35.3644233, 136.6153787),
 (35.3625716, 136.6172645),
 (35.375565, 136.6207),
 (34.6797165, 135.5338832),
 (35.3661958, 136.6178395),
 (35.3663734, 136.6194059),
 (35.3648297, 136.617386),
 (35.3640365, 136.6194888),
 (35.3648526, 136.6187082)]